 # Create a Twisted Interface with Commensurate Lattices

Use the `create_commensurate_interface` function to create twisted interfaces between two materials with commensurate lattice matching.

<h2 style="color:green">Usage</h2>

1. Make sure to select Input Materials (in the outer runtime) before running the notebook.
1. Set notebook parameters in cell 1.1. below (or use the default values).
1. Click "Run" > "Run All" to run all cells.
1. Wait for the run to complete (depending on the parameters can take a few min).
1. Scroll down to view results.

## Summary

1. Prepare the Environment: Set up the notebook and install packages, preview the input materials
1. Create and visualize the initial materials
1. Create a slab from the input material using `create_slab`
1. Generate twisted interface using `create_commensurate_interface` with the specified parameters
1. Visualize the final twisted interface

## Notes

1. We first create a slab from the input material using specified Miller indices.
1. The `create_commensurate_interface` function performs commensurate lattice matching to find valid supercells.
1. The function searches for supercell matrices within specified size limits to achieve the target twist angle.
1. The interface is created with a specified gap between the twisted layers.
1. This approach follows the same pattern as the test suite for reliable results.
1. For more information, see [Introduction](Introduction.ipynb)


## 1. Prepare the Environment
### 1.1. Set up the notebook
Set the following flags to control the notebook behavior


In [ ]:
# Material selection and basic parameters
FILM_INDEX = 0  # Index in the list of materials, to access as materials[FILM_INDEX]
SUBSTRATE_INDEX = None  # Can be None to use same material as film

# Twisted interface parameters
TARGET_TWIST_ANGLE = 17.9  # in degrees
INTERFACE_DISTANCE = 3.0  # in Angstroms
INTERFACE_VACUUM = 20.0  # in Angstroms

# Commensurate interface parameters (following the test pattern)
ANGLE_TOLERANCE = 2.5  # in degrees
MAX_SUPERCELL_MATRIX_INT = 6  # Maximum supercell matrix element value
RETURN_FIRST_MATCH = True  # If True, returns first solution within tolerance

# Slab creation parameters
MILLER_INDICES = (0, 0, 1)  # Miller indices for slab creation
NUMBER_OF_LAYERS = 1  # Number of layers in the slab
USE_CONVENTIONAL_CELL = True
USE_ORTHOGONAL_C = True

STACKING_DIRECTION = "z" # Stacking direction for the slab, can be "x", "y", or "z"

# Visualization parameters
SHOW_INTERMEDIATE_STEPS = True
VISUALIZE_REPETITIONS = [3, 3, 1]

### 1.2. Install packages
Explanation is [here](under_the_hood.ipynb#1.2.-Install-packages).

In [ ]:
import sys

if sys.platform == "emscripten":
    import micropip

    await micropip.install("mat3ra-api-examples", deps=False)
    await micropip.install('mat3ra-utils')
    from mat3ra.utils.jupyterlite.packages import install_packages
    await install_packages("")

### 1.3. Load and preview input materials
Explanation is [here](under_the_hood.ipynb#2.-Data-Exchange).

In [ ]:
from utils.jupyterlite import get_materials

materials = get_materials(globals())

 ## 2. Prepare Materials
 ### 2.1. Select and visualize initial materials

In [ ]:
from utils.visualize import visualize_materials

film = materials[FILM_INDEX]
substrate = materials[SUBSTRATE_INDEX] if SUBSTRATE_INDEX is not None else film

if SHOW_INTERMEDIATE_STEPS:
    visualize_materials(film, repetitions=VISUALIZE_REPETITIONS)
    if substrate is not film:
        visualize_materials(substrate, repetitions=VISUALIZE_REPETITIONS)

## 3. Create Twisted Interface
### 3.1. Create slab and commensurate interface


In [ ]:
from mat3ra.esse.models.core.reusable.axis_enum import AxisEnum
from mat3ra.made.tools.helpers import create_slab
from mat3ra.made.tools.helpers import create_interface_commensurate as create_commensurate_interface

# Create slab from the material
slab = create_slab(
    crystal=film,
    miller_indices=MILLER_INDICES,
    use_conventional_cell=USE_CONVENTIONAL_CELL,
    use_orthogonal_c=USE_ORTHOGONAL_C,
    number_of_layers=NUMBER_OF_LAYERS,
    vacuum=0.0,  # No vacuum in the slab initially
)

print(f"Created slab with {len(slab.basis.elements.ids)} atoms")

# Create the commensurate twisted interface using the function from the test
interface = create_commensurate_interface(
    material=slab,
    target_angle=TARGET_TWIST_ANGLE,
    angle_tolerance=ANGLE_TOLERANCE,
    max_repetition_int=MAX_SUPERCELL_MATRIX_INT,
    return_first_match=RETURN_FIRST_MATCH,
    direction=AxisEnum[STACKING_DIRECTION],  # Stacking direction for the interface
    gap=INTERFACE_DISTANCE,  # Gap between layers
)

### 3.2. Interface creation results


In [ ]:
# The interface was created above using create_commensurate_interface
print("\\nInterface created successfully!")
print(f"Target twist angle: {TARGET_TWIST_ANGLE}°")
print(f"Number of atoms: {len(interface.basis.elements.ids)}")
print(f"Interface name: {interface.name}")

# Show intermediate visualization if requested
if SHOW_INTERMEDIATE_STEPS:
    from utils.visualize import visualize_materials
    print("\\nVisualizing the created interface:")
    visualize_materials(interface, repetitions=[1, 1, 1])
    visualize_materials(interface, repetitions=[1, 1, 1], rotation="-90x")

## 4. Preview the selected material
By default, the first interface is selected. You can change the selection by changing the `selected_interface` index.

In [ ]:
# The interface is already created above, so just display final results
selected_interface = interface

print(f"Final Interface Summary:")
print(f"Target angle: {TARGET_TWIST_ANGLE}°")
print(f"Number of atoms: {len(selected_interface.basis.elements.ids)}")
print(f"Cell vectors:\\n{selected_interface.basis.cell.vector_arrays}")

# Final visualization
from utils.visualize import visualize_materials
print("\\nFinal interface visualization:")
visualize_materials(selected_interface, repetitions=[1, 1, 1])
visualize_materials(selected_interface, repetitions=[1, 1, 1], rotation="-90x")

### 5. Pass data to the outside runtime


In [ ]:
from utils.jupyterlite import set_materials

set_materials(selected_interface)